In [ ]:
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
%matplotlib inline

In [ ]:
basepath = r'D:\naip'

In [ ]:
def reclassify1(raster):
    # Reclassfy values
    # 0: No Vegetation, 1: Vegetation
    # Using threshold of 0 for these images, shrub vegetation
    global img
    img = raster
    img[img>0] = 1
    img[img<=0] = 0
    return img

def reclassify2(raster):
    # Reclassfy values - higher NDVI values, 3 classes
    global img
    img = src
    img[img<0.1] = 0
    img[np.where((img >=0.1) & (img <.4))] = 1
    img[img>=.4] = 2
    return img

# Using bins
def ndviBins(raster):
    class_bins = [-1, .1, .4]
    ndvi_reclass = np.digitize(raster, class_bins)
    return ndvi_reclass

def scaledNDVI(raster):
    global img
    img = src
    img = (img + 1) * 127.5
    return img

In [ ]:
for jp2 in glob.glob(os.path.join(basepath, '*.jp2')):
    print(jp2)
    with rasterio.open(jp2) as src:
        red = src.read(1).astype('float64')
        nir = src.read(4).astype('float64')
        #blue = src.read(3).astype('float64')
        src_meta = src.meta
    
        # ndvi calculation
        ndvi = np.where((nir+red) == 0, 0, (nir-red)/(nir+red))
    
    # Reclassify for analysis
    x = reclassifyPresence(ndvi)
    reclassndvi= os.path.join(basepath,jp2[-39:-4]+'_ndvi_rc.tiff')
    print(reclassndvi)
    
    # Update the metadata dtype and # of bands, output the reclassified file
    new_meta = src.meta
    new_meta.update({"dtype": 'uint8',"count": 1})
    new_meta.update({"count": 1})
    
    with rasterio.open(reclassndvi, 'w', **new_meta) as dst:
        dst.write(x.astype('uint8'),1)
        print(jp2 + ' complete')

In [ ]:
ndvi2 = rasterio.open(reclassndvi)
fig = plt.figure(figsize=(18,12))
plot.show(ndvi2)